# Data Discovery using Redshift Spectrum

## Demonstration of Redshift Capabilities

* Functions:  https://docs.aws.amazon.com/redshift/latest/dg/c_SQL_functions.html
* UDF: https://docs.aws.amazon.com/redshift/latest/dg/user-defined-functions.html
* Store Procedure: https://docs.aws.amazon.com/redshift/latest/dg/stored-procedure-overview.html

Using CMS Data at: https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/Inpatient2016.html

### Author: AWS Professional Services Emerging Technology and Intelligent Platforms Group
### Date: Feb 26 2021



In [ ]:
%reload_ext sql
from aws_orbit_sdk.database import get_redshift
from aws_orbit_sdk.common import get_workspace,get_scratch_database
rs = get_redshift()
workspace = get_workspace()
scratch_glue_db = get_scratch_database()
team_space = workspace['team_space']
env_name = workspace['env_name']
glue_db = f"cms_raw_db_{env_name}".replace('-', '_')
# DO NOT RUN THIS NOTEBOOK IN LAKE CREATOR TEAM SPACE 
assert team_space == 'lake-user'
workspace

#### Start Connection to Redshift (and start cluster if needed)

In [ ]:
rs.get_redshift_functions()

In [ ]:
rs.describe_redshift_function(rs.get_redshift_functions()[0])

In [ ]:
%connect_to_redshift -cluster db-test -reuse -start -func Standard Nodes=3

In [ ]:
%%sql 

SELECT 1 as "Test"

In [ ]:
%create_external_schema -s scratch -g $scratch_glue_db
%create_external_schema -s $glue_db -g $glue_db

In [ ]:
%catalog

In [ ]:
%config SqlMagic.displaylimit=10
%config SqlMagic.short_errors=True

## Start of Actual data discovery code

In [ ]:
%%sql

SELECT tablename,location  FROM SVV_EXTERNAL_TABLES

In [ ]:
%%sql

select * from {glue_db}.beneficiary_summary limit 10

In [ ]:
# Now we can show how you can bind a variable to use within the SQL 

ben_id = "F72554149E321FF9"	

%sql select * from {glue_db}.beneficiary_summary where desynpuf_id = :ben_id

In [ ]:
%%sql 

commit

*** Show DataGrid *** :

In [ ]:
%%sql population_by_age_rs <<
select least(DATE_PART(y,current_date),DATE_PART(y,bene_death_dt)) - DATE_PART(y,bene_birth_dt) as age, 
count(desynpuf_id) as pop_size 
from {glue_db}.beneficiary_summary 
group by age 
order by age


In [ ]:
# Commenting. Missing magic from SDK
#%display_grid population_by_age_rs

In [ ]:
%%create_external_table -g $scratch_glue_db -t myTempTable --a glue_db=$glue_db

select least(DATE_PART(y,current_date),DATE_PART(y,bene_death_dt)) - DATE_PART(y,bene_birth_dt) as age, 
count(desynpuf_id) as pop_size 
from :glue_db.beneficiary_summary 
group by age 
order by age


## Redshift Spectrum Create Table as Select(CTAS)

In [ ]:
%%ddl
DROP TABLE IF EXISTS scratch.myctastable

In [ ]:
# Redshift Spectrum Create Table as Select(CTAS)
ctas_table_name = "myctastable"
scratch_bucket_name = workspace['ScratchBucket']
ctas_table_path = f"{scratch_bucket_name}/{scratch_glue_db}/{ctas_table_name}/"
ctas_table_path


In [ ]:
!aws s3 ls --recursive $ctas_table_path
!aws s3 rm $ctas_table_path --recursive
!aws s3 ls --recursive $ctas_table_path

In [ ]:
%%ddl --a glue_db=$glue_db
CREATE EXTERNAL TABLE scratch.myctastable
STORED AS parquet
LOCATION :ctas_table_path
AS 
select * from :glue_db.beneficiary_summary where desynpuf_id = :ben_id

In [ ]:
%catalog -s scratch -t myctastable

In [ ]:
%%sql ctastable <<

SELECT schemaname, tablename  FROM SVV_EXTERNAL_TABLES where schemaname = 'scratch' and tablename = 'myctastable'

In [ ]:
ctastable

In [ ]:
assert ctastable.DataFrame().at[0,'tablename'] == 'myctastable'

## Redshift supports Store Procedures

In [ ]:
%%sql 

-- Redshift supports Store Procedures. Lets create one:
    
CREATE OR REPLACE PROCEDURE test_sp_notebook(f1 IN int, f2 INOUT varchar(256), OUT varchar(256))
AS $$
DECLARE
  out_var alias for $3;
  loop_var int;
BEGIN
  IF f1 is null OR f2 is null THEN
    RAISE EXCEPTION 'input cannot be null';
  END IF;
  DROP TABLE if exists my_etl;
  CREATE TEMP TABLE my_etl(a int, b varchar);
    FOR loop_var IN 1..f1 LOOP
        insert into my_etl values (loop_var, f2);
        f2 := f2 || '+' || f2;
    END LOOP;
  SELECT INTO out_var count(*) from my_etl;
END;
$$ LANGUAGE plpgsql;

In [ ]:
%%sql

-- lets see if we can call it 

call test_sp_notebook(4,'2019');

In [ ]:
%%sql 

-- Redshift supports temporary tables and Creata Table As for Redshift tables: 

CREATE Temporary TABLE "MyData3" AS (select current_date, * from {glue_db}.beneficiary_summary);

select count(*) from "MyData3";

## This section is for Python and Pandas Programmers

In [ ]:
# Lets see what we got into our variable

population_by_age = population_by_age_rs.DataFrame()
population_by_age.head()

In [ ]:
# Play with visualization:

ax1 = population_by_age.plot.scatter(x='age',
                      y='pop_size',
                       c='DarkBlue')

In [ ]:
%%sql 

-- Now I can use the temporary table

select * from "MyData3" limit 2

In [ ]:
%%sql 

commit

## Lets generate statistical analysis for the data we just created

## Now we can create native Redshift tables and even update them 

In [ ]:
%%sql 
-- Lets create a Redshift Table from the Glue catalog

DROP TABLE IF EXISTS "MyUpdatableTable";

In [ ]:
%%sql 
-- Lets create a Redshift Table from the Glue catalog

CREATE TABLE "MyUpdatableTable"
AS (select 'M' as Gender, * from {glue_db}.beneficiary_summary);


In [ ]:
%%sql 

UPDATE "MyUpdatableTable"
Set gender = case when bene_sex_ident_cd=1 then 'M' else 'F' end
where DATE_PART(y,bene_birth_dt) < 1940

In [ ]:
%%sql data <<

select gender,count(*) as cnt
from  MyUpdatableTable
group by gender

In [ ]:
data

In [ ]:
assert data.DataFrame().at[0,'cnt'] == 214413

In [ ]:
# Deleting redshift cluster
%delete_redshift_cluster -cluster db-test